**재무데이터 활용 최종등급 예측 모델링**



1.   라이브러리 및 데이터 불러오기



In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [26]:
df=pd.read_csv('/content/sampling.csv',sep=",")


2.   데이터 EDA 및 전처리





In [27]:
df.head()

,사업자번호,모형,업종,형태,산업코드,총자산,대표자CB등급,총점,재무등급,최종등급
0,4178131049,401,M,4,2929,297,0,54,B,B
1,6061999434,501,M,8,2524,0,7,53,B,D
2,3030421601,502,M,4,4641,0,7,41,CC,CC
3,1080372906,505,M,8,0,0,7,46,CCC,D
4,6180339705,505,M,8,0,0,0,42,CC,D


In [28]:
df.shape

(110790, 10)

In [29]:
# 문자형-> 숫자형 변환 
df1['재무등급']=df['재무등급'].map({'D':0,'CCC':1,'CC':2,'C':3,'B':4,'BB':5,'BBB':6,'A':7,'AA':8,'AAA':9})
df1['최종등급']=df['최종등급'].map({'D':0,'CCC':1,'CC':2,'C':3,'B':4,'BB':5,'BBB':6,'A':7,'AA':8,'AAA':9})
df1['업종']=df['업종'].map({'M':11,'O':22,'V':33,'S':44,'B':55,'A':66})

In [30]:
# 결측치 제거 
df1=df1.dropna()

In [31]:
# 결측 제거 후 확인 
df1.isna().sum()

사업자번호      0
모형         0
업종         0
형태         0
산업코드       0
총자산        0
대표자CB등급    0
총점         0
재무등급       0
최종등급       0
dtype: int64

In [32]:
df1.head()

,사업자번호,모형,업종,형태,산업코드,총자산,대표자CB등급,총점,재무등급,최종등급
0,4178131049,401,11.0,4,2929,297,0,54,4.0,4.0
1,6061999434,501,11.0,8,2524,0,7,53,4.0,0.0
2,3030421601,502,11.0,4,4641,0,7,41,2.0,2.0
3,1080372906,505,11.0,8,0,0,7,46,1.0,0.0
4,6180339705,505,11.0,8,0,0,0,42,2.0,0.0





3.   모델링



In [34]:
# features/target, train/test dataset 분리
# 사업자번호의 경우 최종등급의 영향이 크지 않을것으로 예측되어 제외
X = df1[['모형','업종','형태','산업코드','총자산','대표자CB등급','총점','재무등급']]
y = df1[['최종등급']]

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 

In [44]:
## 데이터 스케일링
# 데이터 스케일링 작업을 통해, 모든 특성의 범위(또는 분포)를 같게 만들어줘야합니다.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [53]:
## 랜덤포레스트 모델 생성 및 평가 
model_rf = RandomForestClassifier(n_estimators=20, max_depth=5,random_state=0)
model_rf.fit(train_x,train_y)

# SCORE 산출
predict1 = model_rf.predict(test_x)
print(accuracy_score(test_y,predict1))
# 중요도
print("변수 중요도 : \n{}".format(model_rf.feature_importances_))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9815587063293485
변수 중요도 : 
[0.00643352 0.         0.44090507 0.0149924  0.00427392 0.07515954
 0.19403238 0.26420317]


In [51]:
## 로지스틱 모델 생성 및 평가 

model_lg = LogisticRegression()
model_lg.fit(train_x, train_y)

# SCORE 산출
predict2 = model_lg.predict(test_x)
print(accuracy_score(test_y,predict2))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9804930034287832


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [55]:
# 새로운 데이터 예측하기 

jack=np.array([504,11,4,8888,0,2,70,6])
me=np.array([504,11,4,9999,100,4,20,3])
i=np.array([504,11,4,7777,0,2,70,2])

sample_passengers = np.array([jack, me,i])

# 새로운 데이터도 스케일링 해줘야함.
sample_passengers = scaler.transform(sample_passengers)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [57]:
# 새로운 데이터에 대해 최종등급 예측

print(model_rf.predict(sample_passengers))
print(model_lg.predict(sample_passengers))

[6. 2. 4.]
[6. 3. 1.]




4.   제출파일 생성



In [ ]:
predict1 = model.predict(test_x)

In [ ]:
#제출용 파일 infile
submission=pd.read_csv('/content/submission.csv')
submission

In [ ]:
predict1

array([0., 5., 0., ..., 1., 0., 2.])

In [ ]:
submission['최종등급']=predict1
submission

In [ ]:
submission.to_csv('결과물제출.csv')